In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from schemas import DataCatalog
import constants as const
from ons_population import ONSPopulationScenario
import seaborn as sns

In [7]:
#Loading Data-catalog
catalog =  DataCatalog.load_from_yaml("data_catalog.yaml")

## **Loading the Dataset - 'Patients Registered at a GP practice, October 2023'**

In [217]:
#Loading the Dataset to get data on number of patients
gp_patients_reg_catalog_entry = catalog.single_data_sources[6]
gp_patients_reg = gp_patients_reg_catalog_entry.load()
print(gp_patients_reg_catalog_entry.zip_file.namelist())
gp_patients_reg.head()

['gp-reg-pat-prac-quin-age.csv']


/workspaces/PrimaryCareDemandAndCapacity/schemas.py:146: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(csv_file, **read_csv_kwargs)


,PUBLICATION,EXTRACT_DATE,ORG_TYPE,ORG_CODE,ONS_CODE,POSTCODE,SEX,AGE_GROUP_5,NUMBER_OF_PATIENTS
0,GP_PRAC_PAT_LIST,01Oct2023,Comm Region,Y56,E40000003,NaN,ALL,ALL,10927943
1,GP_PRAC_PAT_LIST,01Oct2023,Comm Region,Y56,E40000003,NaN,FEMALE,0_4,258852
2,GP_PRAC_PAT_LIST,01Oct2023,Comm Region,Y56,E40000003,NaN,FEMALE,10_14,300771
3,GP_PRAC_PAT_LIST,01Oct2023,Comm Region,Y56,E40000003,NaN,FEMALE,15_19,287618
4,GP_PRAC_PAT_LIST,01Oct2023,Comm Region,Y56,E40000003,NaN,FEMALE,20_24,385681


In [218]:
print(gp_patients_reg['AGE_GROUP_5'].unique())

['ALL' '0_4' '10_14' '15_19' '20_24' '25_29' '30_34' '35_39' '40_44'
 '45_49' '50_54' '55_59' '5_9' '60_64' '65_69' '70_74' '75_79' '80_84'
 '85_89' '90_94' '95+']


In [219]:
#Replacing AGE_BANDS values to match the ONS_projection AGE_BANDS
gp_patients_reg['AGE_GROUP_5'] = gp_patients_reg['AGE_GROUP_5'].replace({'90_94':'90+', '95+':'90+', '0_4':'0-4', '5_9':'5-9', '10_14':'10-14', '15_19':'15-19', '20_24':'20-24', '25_29':'25-29',
                                                                         '30_34':'30-34', '35_39':'35-39', '40_44':'40-44', '45_49':'45-49', '50_54':'50-54', '55_59':'55-59', '60_64':'60-64',
                                                                         '65_69':'65-69', '70_74':'70-74', '75_79':'75-79', '80_84':'80-84', '85_89':'85-89'})

#Dropping unused columns
gp_patients_reg = gp_patients_reg.loc[gp_patients_reg['AGE_GROUP_5']!='ALL'].drop(columns=['PUBLICATION','EXTRACT_DATE','ORG_TYPE','POSTCODE','SEX']).set_index('ONS_CODE')

print(gp_patients_reg['AGE_GROUP_5'].unique())
print(gp_patients_reg.shape)
gp_patients_reg.head()

['0-4' '10-14' '15-19' '20-24' '25-29' '30-34' '35-39' '40-44' '45-49'
 '50-54' '55-59' '5-9' '60-64' '65-69' '70-74' '75-79' '80-84' '85-89'
 '90+']
(311080, 3)


,ORG_CODE,AGE_GROUP_5,NUMBER_OF_PATIENTS
ONS_CODE,,,
E40000003,Y56,0-4,258852
E40000003,Y56,10-14,300771
E40000003,Y56,15-19,287618
E40000003,Y56,20-24,385681
E40000003,Y56,25-29,535500


### **Filtering the data for Sub-ICB's under SNEE ICB**

In [270]:
required_icb_ons_code = ['06L', '06T', '07K']
snee_gp_patients_reg = gp_patients_reg[gp_patients_reg['ORG_CODE'].isin(required_icb_ons_code)].groupby(['ONS_CODE','AGE_GROUP_5']).sum().drop(columns=['ORG_CODE'])

snee_gp_patients_reg

NUMBER_OF_PATIENTS
ONS_CODE  AGE_GROUP_5                    
E38000086 0-4                       18148
          10-14                     24231
          15-19                     23513
          20-24                     19718
          25-29                     22975
          30-34                     26197
          35-39                     27046
          40-44                     26757
          45-49                     24675
          5-9                       22318
          50-54                     28246
          55-59                     29474
          60-64                     27504
          65-69                     24094
          70-74                     22465
          75-79                     21514
          80-84                     13002
          85-89                      8418
          90+                        4944
E38000117 0-4                       17141
          10-14                     21753
          15-19                     19603
          20-24                     22057
          25-29                     23587
          30-34                     25308
          35-39                     23991
          40-44                     22694
          45-49                     21090
          5-9                       20542
          50-54                     23563
          55-59                     24997
          60-64                     23198
          65-69                     20141
          70-74                     19583
          75-79                     20080
          80-84                     11895
          85-89                      7241
          90+                        4104
E38000204 0-4                       12369
          10-14                     15470
          15-19                     14609
          20-24                     12434
          25-29                     15253
          30-34                     17380
          35-39                     17559
          40-44                     17408
          45-49                     16273
          5-9                       14201
          50-54                     19191
          55-59                     20598
          60-64                     18664
          65-69                     15853
          70-74                     15371
          75-79                     15362
          80-84                      9427
          85-89                      5679
          90+                        3004

## **Loading the ONS Population projections**

In [271]:
ons_projection_catalog_entry = catalog.scenario_data_sources[0].scenarios[0]
ons_projections = ons_projection_catalog_entry.load()
print(ons_projection_catalog_entry.zip_file.namelist())
print(ons_projections.shape)
ons_projections.head()

['2018 SNPP CCG pop females.csv', '2018 SNPP CCG pop males.csv', '2018 SNPP CCG pop persons.csv', 'SNPP Z2 CCG Pop_read me.txt']
(17940, 31)


,AREA_CODE,AREA_NAME,COMPONENT,SEX,AGE_GROUP,2018,2019,2020,2021,2022,...,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043
0,E38000004,NHS Barking and Dagenham CCG,Population,persons,0,3819.0,3539.476,3553.996,3545.026,3503.164,...,3485.017,3512.290,3542.499,3574.884,3607.783,3638.390,3663.012,3682.648,3697.617,3706.681
1,E38000004,NHS Barking and Dagenham CCG,Population,persons,1,3923.0,3841.214,3570.205,3583.060,3571.184,...,3482.414,3507.074,3534.224,3564.456,3597.080,3630.402,3661.624,3687.005,3707.455,3723.301
2,E38000004,NHS Barking and Dagenham CCG,Population,persons,2,4050.0,3912.292,3826.447,3565.341,3575.791,...,3448.544,3469.903,3494.032,3520.703,3550.533,3582.885,3616.072,3647.352,3672.985,3693.800
3,E38000004,NHS Barking and Dagenham CCG,Population,persons,3,3822.0,4036.658,3906.033,3816.546,3563.662,...,3426.572,3442.075,3462.911,3486.569,3512.830,3542.304,3574.383,3607.421,3638.690,3664.447
4,E38000004,NHS Barking and Dagenham CCG,Population,persons,4,3905.0,3805.264,4003.299,3879.217,3788.087,...,3394.732,3405.045,3420.062,3440.323,3463.422,3489.140,3518.097,3549.706,3582.360,3613.373


In [272]:
print(ons_projections['AGE_GROUP'].unique())

['0' '1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15'
 '16' '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29'
 '30' '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '43'
 '44' '45' '46' '47' '48' '49' '50' '51' '52' '53' '54' '55' '56' '57'
 '58' '59' '60' '61' '62' '63' '64' '65' '66' '67' '68' '69' '70' '71'
 '72' '73' '74' '75' '76' '77' '78' '79' '80' '81' '82' '83' '84' '85'
 '86' '87' '88' '89' '90 and over' 'All ages']


In [273]:
#Filtering the dataset to remove values corresponding to 'All ages' in the 'AGE_GROUP' column
ons_projections_df = ons_projections.loc[ons_projections['AGE_GROUP']!='All ages'].drop(columns=['AREA_NAME','COMPONENT','SEX','2018','2019','2020','2021','2022']).set_index('AREA_CODE')

#Replacing '90 and over' with 90
ons_projections_df['AGE_GROUP'] = ons_projections_df['AGE_GROUP'].replace('90 and over', 90)

### **Appending new column having age bands data**

In [274]:
bins = [-1,4,9,14,19,24,29,34,39,44,49,54,59,64,69,74,79,84,89,float('inf')]
labels = ['0-4','5-9','10-14','15-19','20-24','25-29','30-34','35-39','40-44','45-49','50-54','55-59','60-64','65-69','70-74','75-79','80-84','85-89','90+']

#Converting AGE_GROUP data type to int
ons_projections_df['AGE_GROUP'] = ons_projections_df['AGE_GROUP'].astype(int)

#Adding the column for Age Bands as 'AGE_GROUP_5'
ons_projections_df['AGE_GROUP_5'] = pd.cut(ons_projections_df['AGE_GROUP'], bins=bins, labels=labels)
ons_projections_df.head()

,AGE_GROUP,2023,2024,2025,2026,2027,2028,2029,2030,2031,...,2035,2036,2037,2038,2039,2040,2041,2042,2043,AGE_GROUP_5
AREA_CODE,,,,,,,,,,,,,,,,,,,,,
E38000004,0,3469.075,3437.253,3428.219,3418.083,3406.972,3400.285,3399.890,3408.270,3420.751,...,3512.290,3542.499,3574.884,3607.783,3638.390,3663.012,3682.648,3697.617,3706.681,0-4
E38000004,1,3529.219,3494.651,3462.900,3453.457,3443.087,3431.919,3424.981,3424.216,3431.942,...,3507.074,3534.224,3564.456,3597.080,3630.402,3661.624,3687.005,3707.455,3723.301,0-4
E38000004,2,3561.177,3519.204,3484.008,3453.543,3443.862,3433.423,3422.364,3415.261,3414.190,...,3469.903,3494.032,3520.703,3550.533,3582.885,3616.072,3647.352,3672.985,3693.800,0-4
E38000004,3,3572.051,3554.902,3512.813,3478.309,3448.940,3439.117,3428.663,3417.715,3410.517,...,3442.075,3462.911,3486.569,3512.830,3542.304,3574.383,3607.421,3638.690,3664.447,0-4
E38000004,4,3543.565,3549.867,3530.741,3489.970,3456.359,3428.155,3418.331,3407.972,3397.239,...,3405.045,3420.062,3440.323,3463.422,3489.140,3518.097,3549.706,3582.360,3613.373,0-4


### **Filtering the data for Sub-ICB's under SNEE ICB**

In [275]:
#Filtering the data to keep only SNEE ICB
required_icb = ['E38000086','E38000204','E38000117']
snee_ons_projections = ons_projections_df[ons_projections_df.index.isin(required_icb)].groupby(['AREA_CODE', 'AGE_GROUP_5']).sum().drop(columns=['AGE_GROUP'])

/tmp/ipykernel_28262/3081736588.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  snee_ons_projections = ons_projections_df[ons_projections_df.index.isin(required_icb)].groupby(['AREA_CODE', 'AGE_GROUP_5']).sum().drop(columns=['AGE_GROUP'])


### **Calculating the population change factor for each year keeping 2023 as baseline**

In [276]:
pop_baseline = snee_ons_projections['2023']
snee_factors_ons_projections = snee_ons_projections.div(pop_baseline, axis=0)

snee_factors_ons_projections

2023      2024      2025      2026      2027      2028  \
AREA_CODE AGE_GROUP_5                                                           
E38000086 0-4           1.0  0.996380  0.991655  0.987725  0.985527  0.983641   
          5-9           1.0  0.981865  0.965204  0.944608  0.932858  0.923596   
          10-14         1.0  0.996820  0.987534  0.979027  0.966277  0.956538   
          15-19         1.0  1.012988  1.030282  1.043021  1.048398  1.042960   
          20-24         1.0  0.989818  0.990810  1.009767  1.036733  1.065667   
          25-29         1.0  0.989667  0.973707  0.952380  0.924398  0.908342   
          30-34         1.0  0.992156  0.983539  0.970851  0.964521  0.958201   
          35-39         1.0  1.016216  1.018912  1.024337  1.022098  1.011545   
          40-44         1.0  1.000864  1.001372  1.004621  1.010020  1.025212   
          45-49         1.0  0.993796  1.007843  1.029366  1.056115  1.073446   
          50-54         1.0  0.971672  0.942097  0.902643  0.870876  0.845105   
          55-59         1.0  1.000291  0.993214  0.990350  0.977951  0.966341   
          60-64         1.0  1.029504  1.061480  1.076608  1.093181  1.095029   
          65-69         1.0  1.016197  1.036931  1.071070  1.089670  1.115274   
          70-74         1.0  0.993165  0.987046  0.988881  1.009658  1.033405   
          75-79         1.0  1.026945  1.047133  1.055172  1.002303  0.979263   
          80-84         1.0  1.052658  1.109136  1.180786  1.319203  1.391697   
          85-89         1.0  1.030351  1.040814  1.041648  1.074046  1.119122   
          90+           1.0  1.020498  1.053860  1.078494  1.107538  1.145308   
E38000117 0-4           1.0  1.005708  1.009783  1.014158  1.018496  1.022225   
          5-9           1.0  0.988998  0.978590  0.972972  0.968345  0.965322   
          10-14         1.0  1.014354  1.022443  1.024688  1.013315  1.007629   
          15-19         1.0  1.026223  1.057785  1.082351  1.115613  1.131608   
          20-24         1.0  1.000309  1.007522  1.024824  1.052026  1.081884   
          25-29         1.0  0.995608  0.995582  0.985808  0.969342  0.958068   
          30-34         1.0  1.001306  0.987740  0.972500  0.962671  0.959738   
          35-39         1.0  1.033347  1.060788  1.096786  1.120506  1.121589   
          40-44         1.0  1.008458  1.019430  1.028167  1.042682  1.066663   
          45-49         1.0  1.001115  1.010028  1.030459  1.050685  1.074346   
          50-54         1.0  0.973975  0.959241  0.930524  0.907564  0.893369   
          55-59         1.0  1.004293  1.000738  0.993819  0.985711  0.969192   
          60-64         1.0  1.032580  1.057557  1.084425  1.096411  1.109316   
          65-69         1.0  1.025029  1.055317  1.090813  1.132360  1.162132   
          70-74         1.0  0.976653  0.969233  0.974710  0.985601  1.001698   
          75-79         1.0  1.019763  1.027231  1.019696  0.955503  0.926453   
          80-84         1.0  1.058285  1.107336  1.177235  1.322212  1.397038   
          85-89         1.0  1.028112  1.043005  1.039387  1.063122  1.113128   
          90+           1.0  1.007498  1.032613  1.049325  1.073203  1.102895   
E38000204 0-4           1.0  0.997563  0.993782  0.990735  0.989606  0.988545   
          5-9           1.0  0.999140  1.001613  0.987822  0.986740  0.977915   
          10-14         1.0  0.999626  0.991004  0.986916  0.974586  0.966132   
          15-19         1.0  1.014331  1.036693  1.063874  1.076431  1.086404   
          20-24         1.0  0.993080  0.993213  1.006898  1.027327  1.052333   
          25-29         1.0  0.993390  0.981409  0.964646  0.945783  0.933612   
          30-34         1.0  0.996109  0.989536  0.978410  0.976308  0.975134   
          35-39         1.0  1.008305  1.007108  1.009603  1.002256  0.983975   
          40-44         1.0  1.005738  1.014596  1.025005  1.025923  1.044074   
          45-49         1.0  0.995830  1.009596  1.027

## **Combining both the dataframes for 'snee_factors_ons_projections' & 'snee_gp_patients_reg'**

In [282]:
#Using the Inner join to join both the dataframes
snee_factors_vs_patients = snee_factors_ons_projections.join(snee_gp_patients_reg['NUMBER_OF_PATIENTS'])

print(snee_factors_vs_patients.index.nunique())
snee_factors_vs_patients.head(171)

171


2023      2024      2025      2026      2027  \
AGE_GROUP_5 AREA_CODE ONS_CODE                                                  
0-4         E38000086 E38000086   1.0  0.996380  0.991655  0.987725  0.985527   
                      E38000117   1.0  0.996380  0.991655  0.987725  0.985527   
                      E38000204   1.0  0.996380  0.991655  0.987725  0.985527   
            E38000117 E38000086   1.0  1.005708  1.009783  1.014158  1.018496   
                      E38000117   1.0  1.005708  1.009783  1.014158  1.018496   
...                               ...       ...       ...       ...       ...   
90+         E38000117 E38000117   1.0  1.007498  1.032613  1.049325  1.073203   
                      E38000204   1.0  1.007498  1.032613  1.049325  1.073203   
            E38000204 E38000086   1.0  1.026215  1.060871  1.097802  1.136359   
                      E38000117   1.0  1.026215  1.060871  1.097802  1.136359   
                      E38000204   1.0  1.026215  1.060871  1.097802  1.136359   

                                     2028      2029      2030      2031  \
AGE_GROUP_5 AREA_CODE ONS_CODE                                            
0-4         E38000086 E38000086  0.983641  0.982013  0.980844  0.980369   
                      E38000117  0.983641  0.982013  0.980844  0.980369   
                      E38000204  0.983641  0.982013  0.980844  0.980369   
            E38000117 E38000086  1.022225  1.025762  1.029470  1.033490   
                      E38000117  1.022225  1.025762  1.029470  1.033490   
...                                   ...       ...       ...       ...   
90+         E38000117 E38000117  1.102895  1.134087  1.160199  1.162956   
                      E38000204  1.102895  1.134087  1.160199  1.162956   
            E38000204 E38000086  1.180851  1.226857  1.255998  1.278665   
                      E38000117  1.180851  1.226857  1.255998  1.278665   
                      E38000204  1.180851  1.226857  1.255998  1.278665   

                                     2032  ...      2035      2036      2037  \
AGE_GROUP_5 AREA_CODE ONS_CODE             ...                                 
0-4         E38000086 E38000086  0.980962  ...  0.990170  0.995754  1.002608   
                      E38000117  0.980962  ...  0.990170  0.995754  1.002608   
                      E38000204  0.980962  ...  0.990170  0.995754  1.002608   
            E38000117 E38000086  1.038171  ...  1.058531  1.067479  1.077458   
                      E38000117  1.038171  ...  1.058531  1.067479  1.077458   
...                                   ...  ...       ...       ...       ...   
90+         E38000117 E38000117  1.196975  ...  1.407214  1.477103  1.658987   
                      E38000204  1.196975  ...  1.407214  1.477103  1.658987   
            E38000204 E38000086  1.323404  ...  1.549224  1.628954  1.806314   
                      E38000117  1.323404  ...  1.549224  1.628954  1.806314   
                      E38000204  1.323404  ...  1.549224  1.628954  1.806314   

                                     2038      2039      2040      2041  \
AGE_GROUP_5 AREA_CODE ONS_CODE                                            
0-4         E38000086 E38000086  1.010566  1.019359  1.028474  1.037319   
                      E38000117  1.010566  1.019359  1.028474  1.037319   
                      E38000204  1.010566  1.019359  1.028474  1.037319   
            E38000117 E38000086  1.088274  1.099777  1.111485  1.122801   
                      E38000117  1.088274  1.099777  1.111485  1.122801   
...                                   ...       ...       ...       ...   
90+         E38000117 E38000117  1.763368  1.821053  1.846837  1.854606   
                      E38000204  1.763368  1.821053  1.846837  1.854606   
            E38000204 E38000086  1.916266  1.980691  2.020363  2.041737   
                      E38000117  1.916266  1.980691  2.020363  2.041737   
                      E38000204  1.916266  1.980691  2.020363 